# Analyzing the MSTIS simulation

Included in this notebook:

* Opening files for analysis
* Rates, fluxes, total crossing probabilities, and condition transition probabilities
* Per-ensemble properties such as path length distributions and interface crossing probabilities
* Move scheme analysis
* Replica exchange analysis
* Replica move history tree visualization
* Replaying the simulation
* MORE TO COME! Like free energy projections, path density plots, and more

In [1]:
import logging
reload(logging)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info('Info logging activated')
init_log = logging.getLogger('openpathsampling.initialization')
init_log.setLevel(logging.CRITICAL)

INFO:root:Info logging activated


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import openpathsampling as paths
import numpy as np

The optimum way to use storage depends on whether you're doing production or analysis. For analysis, you should open the file as an `AnalysisStorage` object. This makes the analysis much faster.

In [3]:
%%time
storage = paths.AnalysisStorage("mstis.nc")

INFO:openpathsampling.netcdfplus.netcdfplus:Open existing netCDF file 'mstis.nc' for reading - reading from existing file
INFO:openpathsampling.storage.storage:Cached all CVs in 388 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `cvs` [3] in 15 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `samples` [639] in 2369 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `samplesets` [505] in 382 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `volumes` [44] in 0 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `ensembles` [155] in 1 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `pathmovers` [89] in 410 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `pathmovechanges` [2358] in 2537 ms
INFO:openpathsampling.storage.storage:Cache all objects of store `steps` [505] in 446 ms


CPU times: user 6.94 s, sys: 264 ms, total: 7.2 s
Wall time: 7.47 s


In [4]:
storage.pathmovechanges[26].samples[0].trajectory[0].coordinates

array([[ 0.1494084 ,  0.28020981]], dtype=float32)

In [5]:
mstis = storage.networks.load(0)

In [6]:
for store in storage.objects.values():
    print store.name, len(store.index), len(store), store.__class__.__name__

schemes 1 1 NamedObjectStore
[NamedObjectStore] 23 23 NamedObjectStore
snapshots 32044 32044 SnapshotWrapperStore
tag 0 0 ImmutableDictStore
transitions 3 3 NamedObjectStore
networks 1 1 NamedObjectStore
ensembles 155 155 NamedObjectStore
engines 1 1 NamedObjectStore
topologies 1 1 NamedObjectStore
cv2 0 16022 SnapshotValueStore
pathmovers 89 89 NamedObjectStore
cv0 0 16022 SnapshotValueStore
cv1 0 16022 SnapshotValueStore
details 2825 2825 ObjectStore
samples 639 639 SampleStore
trajectories 424 424 TrajectoryStore
snapshot0 16022 16022 FeatureSnapshotStore
pathmovechanges 2358 2358 PathMoveChangeStore
shootingpointselectors 8 8 NamedObjectStore
pathsimulators 1 1 NamedObjectStore
cvs 3 3 CVStore
steps 505 505 MCStepStore
volumes 44 44 NamedObjectStore
samplesets 505 505 SampleSetStore


## Reaction rates

TIS methods are especially good at determining reaction rates, and OPS makes it extremely easy to obtain the rate from a TIS network.

Note that, although you can get the rate directly, it is very important to look at other results of the sampling (illustrated in this notebook and in notebooks referred to herein) in order to check the validity of the rates you obtain.

By default, the built-in analysis calculates histograms the maximum value of some order parameter and the pathlength of every sampled ensemble. You can add other things to this list as well, but you must always specify histogram parameters for these two. The pathlength is in units of frames.

In [7]:
mstis.hist_args['max_lambda'] = { 'bin_width' : 0.02, 'bin_range' : (0.0, 0.5) }
mstis.hist_args['pathlength'] = { 'bin_width' : 5, 'bin_range' : (0, 150) }

In [8]:
cv = storage.cvs[0]

In [9]:
storage.trajectories[200]

Trajectory[89]

In [10]:
len(storage.snapshots.index)

32044

In [11]:
len(storage.snapshot0)

16022

In [12]:
storage.trajectories.cache_all()

In [19]:
%%time
cv(storage.trajectories[378])

CPU times: user 1.32 ms, sys: 535 µs, total: 1.86 ms
Wall time: 1.46 ms


[0.03976452783006533,
 0.040841545016098102,
 0.041202213027843805,
 0.041441858448946288,
 0.041778546518536644,
 0.041752346979445749,
 0.041352843129965934,
 0.04192185507457527,
 0.042852166915101263,
 0.042923270095274399,
 0.043270462526622283,
 0.043848508892836723,
 0.044598626618345377,
 0.046544821884700556,
 0.049676693866862853,
 0.052982394462040394,
 0.054728667683359891,
 0.05504891387195919,
 0.054814206607488536,
 0.055286160249038679,
 0.056435879916645677,
 0.057083816129776317,
 0.058589396563748938,
 0.062044871619492729,
 0.066247324851187339,
 0.070380333956940502,
 0.074845053492256564,
 0.080139806458982998,
 0.086082159755590915,
 0.090826634514188792,
 0.093416568588941679,
 0.09513708067572757,
 0.096733513122600881,
 0.098858434651873564,
 0.10143281289748887,
 0.10278425457887977,
 0.10361160901667724,
 0.10423703526822699,
 0.10378870740503728,
 0.10287640394636462,
 0.10195202354368632,
 0.10115381158315173,
 0.10017665517547386,
 0.099009041064364423,
 

In [20]:
sum(map(len, storage.trajectories))

48474

In [21]:
q = storage.snapshots.all()

In [22]:
len(q)

16022

In [28]:
%%time
_ = cv(q)

CPU times: user 32.5 ms, sys: 11.6 ms, total: 44.1 ms
Wall time: 35.5 ms


In [30]:
%%time
cv[storage.snapshots.all()]

CPU times: user 561 ms, sys: 7.27 ms, total: 568 ms
Wall time: 568 ms


[0.97701914357583786,
 0.034759236768728649,
 0.041382434357364731,
 0.048059494796524847,
 0.054480194166557465,
 0.061480891754069955,
 0.06950933647222679,
 0.076559938799228053,
 0.08103386248519473,
 0.084331688762410567,
 0.087813914808626506,
 0.09182752353208469,
 0.095561217874108539,
 0.097573298240526143,
 0.098467836988701013,
 0.099933508014046213,
 0.1013599790694935,
 0.10283356133517528,
 0.10454142357609966,
 0.10631307952857942,
 0.1085670676357267,
 0.11194947206622353,
 0.11771242225309453,
 0.12572648021894589,
 0.13358261644641509,
 0.14040234793512216,
 0.14629177935203191,
 0.15091102604656814,
 0.1554742610360913,
 0.16031896949045443,
 0.16639709690317064,
 0.17344706675597904,
 0.18027415645996631,
 0.18701775864354869,
 0.19385762006172369,
 0.20091475859095526,
 0.20831695760151275,
 0.21578139357330334,
 0.22456919976870685,
 0.23575486247382527,
 0.24560609442718945,
 0.25323497526916272,
 0.26020380215910544,
 0.2666866647178609,
 0.27269830654935501,
 0

In [34]:
len(storage.snapshots.cache)

3

In [35]:
%%time
mstis.rate_matrix(storage.steps, force=True)

INFO:openpathsampling.analysis.wham:iterations=20diff=2.99328339893e-11
INFO:openpathsampling.analysis.wham:       lnZ=[0.0, -0.1529575046488412, -0.2908150870821239]
INFO:openpathsampling.analysis.wham:iterations=86diff=9.26663190661e-11
INFO:openpathsampling.analysis.wham:       lnZ=[0.0, 1.0873699852334753, 0.8401076276916585]
INFO:openpathsampling.analysis.wham:iterations=5diff=5.15989664191e-11
INFO:openpathsampling.analysis.wham:       lnZ=[0.0, 0.008899735116919117, -0.014391782792535798]
INFO:openpathsampling.analysis.tis_analysis:Rate for B -> C
INFO:openpathsampling.analysis.tis_analysis:outer ensemble: I'face 2 <openpathsampling.ensemble.TISEnsemble object at 0x12c9bb490>
INFO:openpathsampling.analysis.tis_analysis:outer lambda: 0.22
INFO:openpathsampling.analysis.tis_analysis:CTP: 186/505=0.368316831683

INFO:openpathsampling.analysis.tis_analysis:RATE = 0.00370048955247
INFO:openpathsampling.analysis.tis_analysis:flux * outer_tcp * ctp = 0.0202429149798 * 0.496323187449 * 

CPU times: user 2.89 s, sys: 75.2 ms, total: 2.96 s
Wall time: 2.97 s


,"{x|opA(x) in [0.0, 0.04]}","{x|opB(x) in [0.0, 0.04]}","{x|opC(x) in [0.0, 0.04]}"
"{x|opA(x) in [0.0, 0.04]}",NaN,0.00787159,0
"{x|opB(x) in [0.0, 0.04]}",0.00336227,NaN,0.00370049
"{x|opC(x) in [0.0, 0.04]}",0.00406913,0.000481183,NaN


In [36]:
%%time
for t in storage.trajectories:
    q = cv(t)

CPU times: user 1.04 s, sys: 42.5 ms, total: 1.09 s
Wall time: 1.06 s


In [37]:
%%time
for s in storage.snapshots.all().as_proxies():
    cv(s)

CPU times: user 864 ms, sys: 36.9 ms, total: 901 ms
Wall time: 881 ms


In [38]:
%%time
ll = 0
for step in storage.steps:
    sset = step.active # take the sampleset after the move
    for sample in sset:
        ll += len(sample.trajectory)
#        print storage.idx(sample.ensemble),
        
        cv(sample)
        
print ll

951840
CPU times: user 1.3 s, sys: 9.79 ms, total: 1.31 s
Wall time: 1.31 s


In [39]:
%%prun
mstis.rate_matrix(storage.steps, force=True)

INFO:openpathsampling.analysis.wham:iterations=20diff=2.99328339893e-11
INFO:openpathsampling.analysis.wham:       lnZ=[0.0, -0.1529575046488412, -0.2908150870821239]
INFO:openpathsampling.analysis.wham:iterations=86diff=9.26663190661e-11
INFO:openpathsampling.analysis.wham:       lnZ=[0.0, 1.0873699852334753, 0.8401076276916585]
INFO:openpathsampling.analysis.wham:iterations=5diff=5.15989664191e-11
INFO:openpathsampling.analysis.wham:       lnZ=[0.0, 0.008899735116919117, -0.014391782792535798]
INFO:openpathsampling.analysis.tis_analysis:Rate for B -> C
INFO:openpathsampling.analysis.tis_analysis:outer ensemble: I'face 2 <openpathsampling.ensemble.TISEnsemble object at 0x12c9bb490>
INFO:openpathsampling.analysis.tis_analysis:outer lambda: 0.22
INFO:openpathsampling.analysis.tis_analysis:CTP: 186/505=0.368316831683

INFO:openpathsampling.analysis.tis_analysis:RATE = 0.00370048955247
INFO:openpathsampling.analysis.tis_analysis:flux * outer_tcp * ctp = 0.0202429149798 * 0.496323187449 * 

The self-rates (the rate of returning the to initial state) are undefined, and return not-a-number.

The rate is calcuated according to the formula:

$$k_{AB} = \phi_{A,0} P(B|\lambda_m) \prod_{i=0}^{m-1} P(\lambda_{i+1} | \lambda_i)$$

where $\phi_{A,0}$ is the flux from state A through its innermost interface, $P(B|\lambda_m)$ is the conditional transition probability (the probability that a path which crosses the interface at $\lambda_m$ ends in state B), and $\prod_{i=0}^{m-1} P(\lambda_{i+1} | \lambda_i)$ is the total crossing probability. We can look at each of these terms individually.

### Total crossing probability

In [ ]:
stateA = storage.volumes["A"]
stateB = storage.volumes["B"]
stateC = storage.volumes["C"]

In [ ]:
tcp_AB = mstis.transitions[(stateA, stateB)].tcp
tcp_AC = mstis.transitions[(stateA, stateC)].tcp
tcp_BC = mstis.transitions[(stateB, stateC)].tcp
tcp_BA = mstis.transitions[(stateB, stateA)].tcp
tcp_CA = mstis.transitions[(stateC, stateA)].tcp
tcp_CB = mstis.transitions[(stateC, stateB)].tcp

plt.plot(tcp_AB.x, tcp_AB)
plt.plot(tcp_CA.x, tcp_CA)
plt.plot(tcp_BC.x, tcp_BC)
plt.plot(tcp_AC.x, tcp_AC) # same as tcp_AB in MSTIS

We normally look at these on a log scale:

In [ ]:
plt.plot(tcp_AB.x, np.log(tcp_AB))
plt.plot(tcp_CA.x, np.log(tcp_CA))
plt.plot(tcp_BC.x, np.log(tcp_BC))

### Flux

Here we also calculate the flux contribution to each transition. The flux is calculated based on 

In [ ]:
import pandas as pd
flux_matrix = pd.DataFrame(columns=mstis.states, index=mstis.states)
for state_pair in mstis.transitions:
    transition = mstis.transitions[state_pair]
    flux_matrix.set_value(state_pair[0], state_pair[1], transition._flux)

flux_matrix

### Conditional transition probability

In [ ]:
outer_ctp_matrix = pd.DataFrame(columns=mstis.states, index=mstis.states)
for state_pair in mstis.transitions:
    transition = mstis.transitions[state_pair]
    outer_ctp_matrix.set_value(state_pair[0], state_pair[1], transition.ctp[transition.ensembles[-1]])    

outer_ctp_matrix

In [ ]:
ctp_by_interface = pd.DataFrame(index=mstis.transitions)
for state_pair in mstis.transitions:
    transition = mstis.transitions[state_pair]
    for ensemble_i in range(len(transition.ensembles)):
        ctp_by_interface.set_value(
            state_pair, ensemble_i,
            transition.conditional_transition_probability(
                storage.steps,
                transition.ensembles[ensemble_i]
        ))
    
    
ctp_by_interface  

## Path ensemble properties

In [ ]:
hists_A = mstis.transitions[(stateA, stateB)].histograms
hists_B = mstis.transitions[(stateB, stateC)].histograms
hists_C = mstis.transitions[(stateC, stateB)].histograms

### Interface crossing probabilities

We obtain the total crossing probability, shown above, by combining the individual crossing probabilities of 

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['max_lambda']:
        normalized = hist['max_lambda'][ens].normalized()
        plt.plot(normalized.x, normalized)

In [ ]:
# add visualization of the sum

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['max_lambda']:
        reverse_cumulative = hist['max_lambda'][ens].reverse_cumulative()
        plt.plot(reverse_cumulative.x, reverse_cumulative)

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['max_lambda']:
        reverse_cumulative = hist['max_lambda'][ens].reverse_cumulative()
        plt.plot(reverse_cumulative.x, np.log(reverse_cumulative))

### Path length histograms

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['pathlength']:
        normalized = hist['pathlength'][ens].normalized()
        plt.plot(normalized.x, normalized)

In [ ]:
for ens in hists_A['pathlength']:
    normalized = hists_A['pathlength'][ens].normalized()
    plt.plot(normalized.x, normalized)

## Sampling properties

The properties we illustrated above were properties of the path ensembles. If your path ensembles are sufficiently well-sampled, these will never depend on how you sample them.

But to figure out whether you've done a good job of sampling, you often want to look at properties related to the sampling process. OPS also makes these very easy.

### Move scheme analysis

In [ ]:
scheme = storage.schemes[0]

In [ ]:
scheme.move_summary(storage.steps)

In [ ]:
scheme.move_summary(storage.steps, 'shooting')

In [ ]:
scheme.move_summary(storage.steps, 'minus')

In [ ]:
scheme.move_summary(storage.steps, 'repex')

In [ ]:
scheme.move_summary(storage.steps, 'pathreversal')

### Replica exchange sampling

See the notebook `repex_networks.ipynb` for more details on tools to study the convergence of replica exchange. However, a few simple examples are shown here. All of these are analyzed with a separate object, `ReplicaNetwork`.

In [ ]:
repx_net = paths.ReplicaNetwork(scheme, storage.steps)

#### Replica exchange mixing matrix

In [ ]:
repx_net.mixing_matrix()

#### Replica exchange graph

The mixing matrix tells a story of how well various interfaces are connected to other interfaces. The replica exchange graph is essentially a visualization of the mixing matrix (actually, of the transition matrix -- the mixing matrix is a symmetrized version of the transition matrix).

Note: We're still developing better layout tools to visualize these.

In [ ]:
repxG = paths.ReplicaNetworkGraph(repx_net)
repxG.draw('spring')

#### Replica exchange flow

Replica flow is defined as ***TODO***

Flow is designed for calculations where the replica exchange graph is linear, which ours clearly is not. However, we can define the flow over a subset of the interfaces.

### Replica move history tree

In [ ]:
import openpathsampling.visualize as vis
reload(vis)
from IPython.display import SVG

In [ ]:
tree = vis.PathTree(
    storage.steps[0:200],
    vis.ReplicaEvolution(replica=2, accepted=False)
)

SVG(tree.svg())

In [ ]:
decorrelated = tree.generator.decorrelated
print "We have " + str(len(decorrelated)) + " decorrelated trajectories."

### Visualizing trajectories

In [ ]:
from toy_plot_helpers import ToyPlot
background = ToyPlot()
background.contour_range = np.arange(-1.5, 1.0, 0.1)
background.add_pes(storage.engines[0].pes)

In [ ]:
xval = paths.CV_Function("xval", lambda snap : snap.xyz[0][0])
yval = paths.CV_Function("yval", lambda snap : snap.xyz[0][1])
live_vis = paths.LiveVisualization(mstis, xval, yval, [-1.0, 1.0], [-1.0, 1.0])
live_vis.background = background.plot()

In [ ]:
live_vis.draw_samples(list(tree.samples))

## Histogramming data (TODO)

In [ ]:
#! skip
# The skip directive tells our test runner not to run this cell
import time
max_step = 10
for step in storage.steps[0:max_step]:
    live_vis.draw_ipynb(step)
    time.sleep(0.1)